A hydrostatic, inviscid, adiabatic, and moist system in isentropic coordinates 
==

In [ ]:
from datetime import datetime, timedelta
import gridtools as gt
import numpy as np
import os
import sympl
import tasmania as taz

# For interactive plotting
from IPython import display
%matplotlib nbagg

Load the computational grid and the initial state
--

In [ ]:
domain_x, nx = sympl.DataArray([0., 500.], dims='x', attrs={'units': 'km'}).to_units('m'), 101
domain_y, ny = sympl.DataArray([-1., 1.], dims='y', attrs={'units': 'km'}).to_units('m'), 1
domain_z, nz = sympl.DataArray([340., 280.], dims='air_potential_temperature', attrs={'units': 'K'}), 60

grid = taz.GridXYZ(domain_x, nx, domain_y, ny, domain_z, nz,
                   topo_type='gaussian', topo_time=timedelta(seconds=1800),
                   topo_kwargs={'topo_max_height': sympl.DataArray(1000.0, attrs={'units': 'm'}),
                                'topo_width_x': sympl.DataArray(25.0, attrs={'units': 'km'})})

time          = datetime(year=1992, month=2, day=20)
x_velocity    = sympl.DataArray(15., attrs={'units': 'm s^-1'})
y_velocity    = sympl.DataArray(0., attrs={'units': 'm s^-1'})
brunt_vaisala = sympl.DataArray(0.01, attrs={'units': 's^-1'})

state = taz.get_default_isentropic_state(grid, time, x_velocity, y_velocity, brunt_vaisala, 
                                         moist_on=True, dtype=np.float64)

# Enrich the state with the accumulated and instantaneous precipitation
state['accumulated_precipitation'] = taz.make_data_array_3d(
    np.zeros((grid.nx, grid.ny, 1)), grid, 'mm')
state['precipitation'] = taz.make_data_array_3d(
    np.zeros((grid.nx, grid.ny, 1)), grid, 'mm h^-1')

Initialize the microphysical parameterizations
--

In [ ]:
# The component calculating the microphysical sources/sinks 
# prescribed by the Kessler scheme; do consider the evaporation of
# rain in the subcloud layers
a = sympl.DataArray(0.1, attrs={'units': 'g kg^-1'})
kessler = taz.Kessler(grid, air_pressure_on_interface_levels=True,
                      rain_evaporation=True, backend=gt.mode.NUMPY,
                      autoconversion_threshold=a)

# The component calculating the raindrop fall velocity
rfv = taz.RaindropFallVelocity(grid, backend=gt.mode.NUMPY)

# The component performing the saturation adjustment 
# as prescribed by the Kessler scheme
saturation = taz.SaturationAdjustmentKessler(grid, air_pressure_on_interface_levels=True,
                                             backend=gt.mode.NUMPY)

Instantiate the dynamical core
--

In [ ]:
# Adopt the flux form of the leapfrog time integration method,
# with the horizontal numerical fluxes calculated via a centered scheme;
# the sedimentation flux is discretized with the help of the second-order upwind scheme
dycore = taz.IsentropicDynamicalCore(grid, moist_on=True,
                                     time_integration_scheme='centered',
                                     horizontal_flux_scheme='centered',
                                     horizontal_boundary_type='relaxed',
                                     smooth_on=True, smooth_type='first_order',
                                     smooth_coeff=0.20, smooth_at_every_stage=True,
                                     adiabatic_flow=True, sedimentation_on=True,
                                     sedimentation_flux_type='second_order_upwind',
                                     sedimentation_substeps=2,
                                     raindrop_fall_velocity_diagnostic=rfv,
                                     backend=gt.mode.NUMPY, dtype=np.float64)

Create a monitor for interactive visualization
---------------------------------------------------------------

In [ ]:
# The drawer and the artist generating the left subplot
drawer1_properties = {
    'fontsize': 16, 'cmap_name': 'Blues', 'cbar_on': True, 
    'cbar_levels': 18, 'cbar_ticks_step': 4, 
    'cbar_orientation': 'horizontal',
    'draw_vertical_levels': True,
}
drawer1 = taz.Contourf(
    grid, 'mass_fraction_of_cloud_liquid_water_in_air', 'g kg^-1', y=0,  
    xaxis_units='km', zaxis_name='height', zaxis_units='km',
    properties=drawer1_properties,
)
topo_drawer = taz.LineProfile(
    grid, 'topography', 'km', y=0, z=-1, axis_units='km',
    properties={'linecolor': 'black', 'linewidth': 1.3}
)
axes1_properties = {
    'fontsize': 16, 'title_left': 'Cloud liquid water [g kg$^{-1}$]',
    'x_label': '$x$ [km]', 'x_lim': [0, 500],
    'y_label': '$z$ [km]', 'y_lim': [0, 14],
}
plot1 = taz.Plot((drawer1, topo_drawer), axes_properties=axes1_properties)

# The drawers and the artist generating the right subplot
drawer2_properties = {
    'fontsize': 16, 'linecolor': 'blue', 'linewidth': 1.5,
    'legend_label': 'Precipitation [mm h$^{-1}$]'
}
drawer2 = taz.LineProfile(
    grid, 'precipitation', 'mm h^-1',
    y=0, z=0, axis_units='km', properties=drawer2_properties,
)
drawer3_properties = {
    'fontsize': 16, 'linecolor': 'red', 'linewidth': 1.5,
    'legend_label': 'Accumulated precipitation [mm]'
}
drawer3 = taz.LineProfile(
    grid, 'accumulated_precipitation', 'mm',
    y=0, z=0, axis_units='km', properties=drawer3_properties,
)
axes2_properties = {
    'fontsize': 16, 'x_label': '$x$ [km]', 'x_lim': [0, 500], 'y_lim': [0, 2],
    'legend_on': True, 'legend_framealpha': 1.0, 'grid_on': True,
}
plot2 = taz.Plot((drawer2, drawer3), axes_properties=axes2_properties)

# The monitor encompassing and coordinating the two artists
figure_properties = {'fontsize': 16, 'figsize': (12, 7), 'tight_layout': True}
plot_monitor = taz.PlotComposite(
    nrows=1, ncols=2, artists=(plot1, plot2), interactive=False, figure_properties=figure_properties
)

Create a monitor to write the solution into a NetCDF file
--

In [ ]:
filename = '../tests/baseline_datasets/isentropic_moist_sedimentation_evaporation.nc'
if os.path.exists(filename):
    os.remove(filename)
netcdf_monitor = taz.NetCDFMonitor(filename, grid)
netcdf_monitor.store(state)

Run the simulation
---------------------------

In [ ]:
# Simulation settings
timestep = timedelta(seconds=10)
niter    = 2160

for i in range(niter):
    # Update the (time-dependent) topography
    dycore.update_topography((i+1)*timestep)
    
    # Calculate the microphysical tendencies
    tendencies, _ = kessler(state)
    
    # Step the solution
    state_new = dycore(state, tendencies, timestep)
    state.update(state_new)
    
    # Perform the saturation adjustment
    state_new = saturation(state)
    state.update(state_new)

    if (i+1) % 60 == 0:
        # Plot the solution
        plot1.axes_properties['title_right'] = str((i+1)*timestep)
        plot2.axes_properties['title_right'] = str((i+1)*timestep)
        fig = plot_monitor.store(((state, state), (state, state)), show=False)
        display.clear_output(wait=True)
        display.display(fig)
        
        # Cache the solution
        netcdf_monitor.store(state)
            
# Write chached states to file
netcdf_monitor.write()

print('Simulation successfully completed. HOORAY!')